<a href="https://colab.research.google.com/github/weibb123/AB-Test-with-essential-concepts/blob/main/AB_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Suppose we have a dataset that is in control group and treatment group with different landing page. We want to run an experiment that randomly split user into one of the group. 

Control group -> user with old page

Treatment group -> user with new page

We want to figure out if new page has a significantly increase in user conversion after being exposed to new vs old page.




# Import

In [1]:
import pandas as pd
from scipy import stats
from scipy.stats import t
import numpy as np
import statsmodels.api as sm
import statsmodels.stats.power as smp

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/patiegm/Udacity_Data_Analysis_Nanodegree/master/Analyze%20AB%20Test%20Results/ab_data.csv")
data.head(5)

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


# Data Processing

In [3]:
def check_mixed(df):
  df1 = df[['user_id', 'group']].groupby(['user_id']).nunique().reset_index() # count unique number of groups that a user was assigned to
  df2 = df1[df1['group'] >1]['user_id'].count()  # count number of users assigned to both group 
  print(df2)

check_mixed(data)

1895


In [4]:
def remove_mixed(df):
  df1 = df[['user_id', 'group']].groupby(['user_id']).nunique().reset_index() # count the unique number of groups that a user was assigned to
  df2 = pd.merge(df, df1, on=['user_id'], how='left') # select all rows from left table and matched values between 2 df
  return df2[df2['group_y'] == 1][['user_id', 'timestamp', 'group_x', 'landing_page', 'converted']]\
  .rename(columns={'group_x':'group'}) # only return users assigned to either treatment or control

In [5]:
data1 = remove_mixed(data)

In [6]:
check_mixed(data1)

0


In [7]:
# make sure to count number of users exposed to treatment OR
# numer of users exposed to control. 
# we want to make sure users are independent of interference since this is essential in A/B testing to prevent overestimate or underestimate

def check_interfere(df):
  # count number of users in control(old_page) exposed to treatment(new_page)
  print(df[(df['group'] == 'control')&(df['landing_page'] == 'new_page')]['user_id'].count())
  # count number of users in treatment(new_page) exposed to control(old_page)
  print(df[(df['group'] == 'treatment')&(df['landing_page'] == 'old_page')]['user_id'].count())

In [8]:
check_interfere(data1)

1007
991


In [9]:
def remove_interfere(df):
  # identify users in control(old_page) exposed to treatment(new_page)
  df1 = df[(df['group'] == 'control')&(df['landing_page'] == 'new_page')][['user_id', 'group']]
  # identify users in treatment(new_page) exposed to control(old_page)
  df2 = df[(df['group'] == 'treatment')&(df['landing_page'] == 'old_page')][['user_id', 'group']]
  
  # concat the dataframes together
  df3 = pd.concat([df1, df2])
  df4 = pd.merge(df, df3, on=['user_id'], how='left')
  return df4[df4['group_y'].isna()][['user_id', 'timestamp', 'group_x', 'landing_page', 'converted']]\
  .rename(columns={'group_x': 'group'})
  # only return users with the correct exposure

In [10]:
data2 = remove_interfere(data1)

In [11]:
check_interfere(data2)

0
0


In [12]:
def check_multi_exposes(df):
  # count the number of exposures that a user had
  df1 = df[['user_id', 'group']].groupby(['user_id']).count().reset_index()
  # count the number of users that had multiple exposures
  df2 = df1[df1['group'] > 1]['user_id'].count()

  print(df2)


In [13]:
check_multi_exposes(data2)

1


In [14]:
def deal_with_multiple_exposes(df):
  # get timestamps of first and last exposure, number of exposure and number of conversions
  df1 = df.groupby(['user_id', 'group', 'landing_page'])\
    .agg({'timestamp': ['min', 'max'], 'converted': ['count', 'sum']})
  
  df1.columns = df1.columns.droplevel(0)
  df2 = df1.reset_index()
  df2['converted'] = df2.apply(lambda x: int(x['sum'] > 0), axis=1) # 1 if user has 1 conversion
  df2['conversion_rate'] = 1.0*df2['sum']/df2['count'] # number of conversions divided by number of exposures
  return df2 # one user will only have 1 row

In [15]:
data3 = deal_with_multiple_exposes(data2)

In [16]:
check_multi_exposes(data3)

0


In [17]:
## Sanity check
def check_imbalance(df):
  # count number of users in treatment vs. control
  df1 = df[['user_id', 'group']].groupby(['group']).count().reset_index()
  print(df1)
  pvalue = stats.binomtest(df1[df1['group'] == 'treatment']['user_id'].values[0],
                            n=df1['user_id'].sum(),
                            p=0.5,
                            alternative='two-sided')
  # test whether treatment vs. control has equal sample size
  print(pvalue)

In [18]:
check_imbalance(data3)

       group  user_id
0    control   143293
1  treatment   143398
BinomTestResult(k=143398, n=286691, alternative='two-sided', statistic=0.500183123990638, pvalue=0.8459923345444519)


# Hypothesis Testing

In [23]:
def calculate_pvalue(df):

  n_treatment = df[df['group'] == 'treatment']['user_id'].count() # get number of users in treatment
  n_control = df[df['group'] == 'control']['user_id'].count() # number of users in control

  # probability of a user in treatment to convert
  p_treatment = 1.0*df[df['group'] == 'treatment']['converted'].sum()/ n_treatment

  # probability of a user in control to convert
  p_control = 1.0*df[df['group'] == 'control']['converted'].sum() / n_control

  # variance of probability of a user in treatment to convert
  var_treatment = p_treatment*(1-p_treatment) # p(1-p)

  # variance of probability of a user in control to convert
  var_control = p_control*(1-p_control) # p(1-p)

  # delta of a probability that user convert in treatment vs. control
  p_delta = p_treatment - p_control 
  print(f"this is p_delta {p_delta}")

  # pooled standard error of t-test
  pooled_se = np.sqrt(var_treatment/n_treatment + var_control/n_control)

  # t-statistic
  t_stat = p_delta / pooled_se

  dof = (var_treatment/n_treatment + var_control/n_control)**2\
        / (var_treatment**2/(n_treatment**2*(n_treatment-1)) + var_control**2/(n_control**2*(n_control-1)))
  
  # p value of t-test
  p_value = 2*t.cdf(-abs(t_stat), dof)
  print(f"This is p_value {p_value}")

  # lower bound of confidence interval
  lower = p_delta - t.ppf(0.975, dof)*pooled_se

  # upper bound of confidence interval
  upper = p_delta + t.ppf(0.975, dof)*pooled_se

  print(f"lower bound {lower}")
  print(f"upper bound {upper}")



In [24]:
calculate_pvalue(data3)

this is p_delta -0.0014478458686042056
This is p_value 0.23202039224946788
lower bound -0.0038221778568021604
upper bound 0.0009264861195937492


# Power Calculation

In power analysis we can compute minimum detectable effect

In [29]:
def cal_mde(df):
  n_treatment = df[df['group'] == 'treatment']['user_id'].count() # get number of users in treatment
  n_control = df[df['group'] == 'control']['user_id'].count() # number of users in control

  power_analysis = smp.TTestIndPower()

  effect_size = power_analysis.solve_power(
      nobs1=n_control, power=0.8, alpha=0.05, ratio=1.0*n_treatment/n_control, alternative='two-sided'
  )

  p = 1.0*df['converted'].sum() / df['user_id'].count() # probability of a user to convert
  sd = np.sqrt(p*(1-p)) # standard deviation of user to convert
  mde = effect_size*sd # minimum detectable effect with the current sample size
  print(f"this is minimum detectable effect {mde}")

  # probability of user in treatment to convert
  p_treatment = 1.0*df[df['group'] == 'treatment']['converted'].sum() / n_treatment

  # probability of user in control to convert
  p_control = 1.0*df[df['group'] == 'control']['converted'].sum() / n_treatment

  # measured delta of the probability of a user to convert in treatment vs. control
  p_delta = p_treatment - p_control

  # the required sample size when setting the currently measured delta as the minimum detectable effect
  n = power_analysis.solve_power(
        effect_size=1.0*p_delta/sd, power=0.8, alpha=0.05, ratio=1.0, alternative='two-sided'
    )
  
  print(f"required sample size {n}")


In [30]:
cal_mde(data3)

this is minimum detectable effect 0.0033939097036790812
required sample size 892876.8943090558


# validation

In [31]:
X = pd.get_dummies(data3['group'], drop_first=True)
y = data3['converted']

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:              converted   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.428
Date:                Mon, 13 Mar 2023   Prob (F-statistic):              0.232
Time:                        05:17:37   Log-Likelihood:                -83972.
No. Observations:              286691   AIC:                         1.679e+05
Df Residuals:                  286689   BIC:                         1.680e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1202      0.001    140.266      0.0